<a href="https://colab.research.google.com/github/pndang/USP_138_Project/blob/main/USP_138_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [19]:
columns = ['2017 NAICS code (NAICS2017)',
           'Meaning of NAICS code (NAICS2017_LABEL)',
           'Meaning of Employment size of establishments code (EMPSZES_LABEL)',
           'Number of employees (EMP)']

df = pd.read_csv("/content/drive/MyDrive/USP_138/SF_Industries.csv", \
                 usecols=columns)

df_3d = pd.read_csv("/content/drive/MyDrive/USP_138/SF_manufacturing_3digitsNAICS.csv", \
                    usecols=columns)

df.head()

,2017 NAICS code (NAICS2017),Meaning of NAICS code (NAICS2017_LABEL),Meaning of Employment size of establishments code (EMPSZES_LABEL),Number of employees (EMP)
0,00,Total for all sectors,All establishments,"720,508"
1,00,Total for all sectors,Establishments with less than 5 employees,N
2,00,Total for all sectors,Establishments with 5 to 9 employees,N
3,00,Total for all sectors,Establishments with 10 to 19 employees,N
4,00,Total for all sectors,Establishments with 20 to 49 employees,N


In [20]:
df_3d.head()

,2017 NAICS code (NAICS2017),Meaning of NAICS code (NAICS2017_LABEL),Meaning of Employment size of establishments code (EMPSZES_LABEL),Number of employees (EMP)
0,311,Food manufacturing,All establishments,"2,052"
1,311,Food manufacturing,Establishments with less than 5 employees,N
2,311,Food manufacturing,Establishments with 5 to 9 employees,N
3,311,Food manufacturing,Establishments with 10 to 19 employees,N
4,311,Food manufacturing,Establishments with 20 to 49 employees,N


In [21]:
rename_dict = {'2017 NAICS code (NAICS2017)': 'NAICS Code',
                   'Meaning of NAICS code (NAICS2017_LABEL)': 'Industry',
                   'Number of employees (EMP)': 'Number of Employees'}

df.rename(columns=rename_dict, inplace=True)
df_3d.rename(columns=rename_dict, inplace=True)

In [22]:
def adjust_format_noe(row):
  noe = str(row['Number of Employees'])
  noe = noe[::-1]
  output = ''

  for i in range(0, len(noe), 3):
    output = output+','+noe[i:i+3]

  return output[::-1].strip(',')


In [23]:
# Pipeline to with steps to clean data

def clean_pl(data):
  data = data[data['Meaning of Employment size of establishments code (EMPSZES_LABEL)'] == 'All establishments']
  data.drop(columns=[x for x in data.columns if 'Meaning' in x], inplace=True)

  data['Number of Employees'] = data['Number of Employees'].str.replace(',','').astype(int)

  total = data['Number of Employees'][0]
  data['Percent'] = (data['Number of Employees'] / total)*100

  data['Percent'] = np.round(data['Percent'], 2)

  data.set_index('NAICS Code', inplace=True)

  data.sort_values(by='Percent', ascending=False, inplace=True)

  data['Number of Employees'] = data.apply(adjust_format_noe, axis=1)

  data['Percent'] = data['Percent'].astype(str)+'%'
  return data

In [24]:
df = clean_pl(df)
df_3d = clean_pl(df_3d)

In [25]:
df

,Industry,Number of Employees,Percent
NAICS Code,,,
00,Total for all sectors,"720,508",100.0%
54,"Professional, scientific, and technical services","134,173",18.62%
51,Information,"87,285",12.11%
72,Accommodation and food services,"86,099",11.95%
62,Health care and social assistance,"74,186",10.3%
52,Finance and insurance,"64,272",8.92%
44-45,Retail trade,"49,806",6.91%
56,Administrative and support and waste managemen...,"41,224",5.72%
55,Management of companies and enterprises,"28,879",4.01%


In [27]:
df_3d.iloc[:15]

,Industry,Number of Employees,Percent
NAICS Code,,,
311,Food manufacturing,"2,052",100.0%
334,Computer and electronic product manufacturing,"1,231",59.99%
312,Beverage and tobacco product manufacturing,813,39.62%
323,Printing and related support activities,724,35.28%
315,Apparel manufacturing,483,23.54%
339,Miscellaneous manufacturing,426,20.76%
327,Nonmetallic mineral product manufacturing,318,15.5%
337,Furniture and related product manufacturing,308,15.01%
332,Fabricated metal product manufacturing,274,13.35%
